# Numpy ArrayとUnnecessary array copying
## Agenda

- Numpy.arrayを用いた処理のときに、省メモリな書き方を実現するために必要な基礎知識を紹介
- `id()`を用いてメモリにおけるローケーションを確認する

### Hardware

In [1]:
%%bash
system_profiler SPHardwareDataType | grep -E \
"Model Identifier"\|"Processor Name"\|"Processor Speed"\
\|"Number of Processors"\|"Memory:"

      Model Identifier: MacBookPro13,1
      Processor Name: Dual-Core Intel Core i5
      Processor Speed: 2 GHz
      Number of Processors: 1
      Memory: 16 GB


### Python

In [2]:
!python -V

Python 3.7.4


### Import

In [3]:
import numpy as np

## 1. Numpy Arrayとメモリロケーション

In [4]:
a = np.array([1, 2])
a_loc = id(a)
print(a_loc)

4542894448


In [5]:
b = a
b_loc = id(b)
a_loc == b_loc

True

In [6]:
b[1] = 10
a

array([ 1, 10])

### Objectのcopy

In [7]:
c = a.copy()
id(c) == id(a)

False

### in-place operationsとimplicit-copy operation

In [8]:
a = np.array([i for i in range(10)])
a_loc = id(a)

In [9]:
a *= 2
id(a) == a_loc

True

一方、

In [10]:
## implicit-copy operation
c = a * 2
id(c) == a_loc

False

この差は計算時間の差に直結する。実際に、

In [11]:
%%timeit

a = np.ones(10000000)
a*=2

21.6 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit

a = np.ones(10000000)
c = a*2

32.1 ms ± 788 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### `reshape`

In [13]:
a = np.zeros((1000, 1000))
a_loc = id(a)
b = a.reshape((1, -1))
id(b) == a_loc

False

### `numpy.array.__array_interface__['data']`とvalueの格納場所

`numpy.array.__array_interface__['data'][0]`を用いることでdata valueがどこに格納されているか判断することができる。

In [14]:
a = np.zeros((1000, 1000))
a_loc = a.__array_interface__['data'][0]
b = a.reshape((1, -1))
b_loc = b.__array_interface__['data'][0]
a_loc == b_loc

True

なので`b`の一つの要素を変更すると`a`の要素も変更される：

In [15]:
a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
b[0, 1] = 10

In [17]:
a

array([[ 0., 10.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### `Transpose` and `Reshape`

transposeのみの場合は

In [18]:
a = np.zeros((1000, 1000))
a_loc = a.__array_interface__['data'][0]
b = a.T
b_loc = b.__array_interface__['data'][0]
a_loc == b_loc

True

In [19]:
a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
b[0, 1] = 10

In [21]:
a

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [10.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

一方、

In [22]:
a = np.zeros((1000, 1000))
a_loc = a.__array_interface__['data'][0]
b = a.T.reshape((1, -1))
b_loc = b.__array_interface__['data'][0]
a_loc == b_loc

False

In [23]:
a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
b[0, 1] = 10

In [25]:
a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

よって実行時間にも大きな差がつく

In [26]:
%%timeit
a = np.zeros((1000, 1000))
b = a.reshape((1, -1))

353 µs ± 6.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
%%timeit
a = np.zeros((1000, 1000))
b = a.T.reshape((1, -1))

2.74 ms ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### `flatten()`と`ravel()`の差異

- どちらもnumpy.arrayを1D arrayに変換する関数
- `flatten()`は常にcopyを返すが、`ravel()`は必要な時しかcopyを返さない

In [28]:
a = np.zeros((1000, 1000))
a_loc = a.__array_interface__['data'][0]
e = a.flatten()
a_loc == e.__array_interface__['data'][0]

False

In [29]:
a = np.zeros((1000, 1000))
a_loc = a.__array_interface__['data'][0]
e = a.ravel()
a_loc == e.__array_interface__['data'][0]

True

### REMARKS: なぜnumpyは速いのか？

- numpy.arrayはlistなどPythonが提供しているその他のdata型と異なり、RAMにおける変数格納領域がdata bufferという形でblock単位で確保されている。
- listなどPythonが提供しているその他のdata型は、変数の値をRAMのあちこちのアドレスを使って格納している
- memory access pattern, CPU cache, vectorized instructionの観点からnumpyが採用しているデータ格納方法はメモリ効率的であるとされている。


## 2. in-place operationとimplicit-copyの差異

- `a *= 2`: in-place operation
- `a = a*2`: implicit-copy operation, `a*2`を格納したarrayがまず新しく作られる。その後、参照されなくなった古い`a`はgarbage collecterによって消去される

In [30]:
## in-place operation
a = np.array([2])
a_loc = a.__array_interface__['data'][0]
a *= 2
a_loc == a.__array_interface__['data'][0]

True

In [31]:
# implicit copy
a = np.array([2])
a_loc = a.__array_interface__['data'][0]
a = a*2
a_loc == a.__array_interface__['data'][0]

False

### Row-majour orderとColumn-major order

<img src = 'https://github.com/RyoNakagami/omorikaizuka/blob/master/algorithm/row_column_major.jpg?raw=true'>


- 行列をプログラム内でデータ保持する場合，データアクセスの効率を考慮してメモリの連続領域にデータが置かれるのが多数だが、そのデータの順が，行列において行と列のどちらが優先方向になるかによってRow-majour orderとColumn-major orderに分類される
- numpyはRow-majour orderを採用している

2D numpy arrayを1Dに変換する時、データのaddressの順番を変更ことなく1Dに変換することができるが、いったん転置を実行してからだと、addressのスキャン順番が飛び飛びになってしまい、実行時間が遅くなり、また最終的にcopyが必要となる。

In [32]:
a = np.arange(100*100).reshape(100, 100)
a

array([[   0,    1,    2, ...,   97,   98,   99],
       [ 100,  101,  102, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  297,  298,  299],
       ...,
       [9700, 9701, 9702, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9897, 9898, 9899],
       [9900, 9901, 9902, ..., 9997, 9998, 9999]])

メモリの移動距離(byte単位)を表示してみると

In [33]:
a.strides

(800, 8)

`base` attributeを用いてndarrayのベースとなるオブジェクト(オリジナル)を表示してみると、

In [34]:
a.base

array([   0,    1,    2, ..., 9997, 9998, 9999])

実行時間の差は以下の例ではrobustな形では確認されなかった

In [35]:
%%timeit
a = np.arange(1000*1000).reshape(1000, 1000)
np.sum(a, axis = 0)

1.16 ms ± 7.74 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
%%timeit
a = np.arange(1000*1000).reshape(1000, 1000)
np.sum(a, axis = 1)

1.06 ms ± 6.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### column major orderで定義した場合のstridesは

In [37]:
b0 = list(np.arange(100*100).reshape(100, 100))
b = np.array(b0, order = 'F') # FはFortranの順序、Column major
b.strides

(8, 800)

### Row-majour orderとColumn-major orderとin-place operationとimplicit-copy

row major orderで定義されたnumpy.ndarrayを一次元に直してみる

In [38]:
c = a.ravel()

In [39]:
id(c) == id(a)

False

In [40]:
c.__array_interface__['data'][0] == a.__array_interface__['data'][0]

True

column major orderで定義されたnumpy.ndarrayを一次元に直してみる

In [41]:
b0 = list(np.arange(100*100).reshape(100, 100))
b = np.array(b0, order = 'F')

In [42]:
d = b.ravel()
d

array([   0,    1,    2, ..., 9997, 9998, 9999])

In [43]:
b.__array_interface__['data'][0] == d.__array_interface__['data'][0]

False

一次元配列に直すときそれぞれの要素の参照メモリアドレスの順番を変えなくてはならないためcopyが生成される

In [44]:
%%timeit
c = a.ravel()

248 ns ± 1.48 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [45]:
%%timeit
d = b.ravel()

9.69 µs ± 879 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


そのため、上の例のように実行時間に差がつく